## Class Decorators

### class decorators
* allows to programmatically transform class definitions
* similar mechanism to function decorators
* metaprgramming - treating progams as data
* overlap with the capabilities of metaclass
* less powerful than metaclass, but easier to use
* class decorators often introspect the decorated class
* class decorators are applied when the decorated class is first being defined, that happens when the module containg the class definition is first imported
* modlues in Python are singletons. Each module only exists once in the memory of a given process, so importing them again has no effects

In [9]:
class Position:
    
    def __init__(self, latitude, longitude):
        if not (-90 <= latitude <= 90):
            raise ValueError(f"Latitude {latitude} out of range")
            
        if not (-180 <= longitude <= 180):
            raise ValueError(f"Longitude {longitude} out of range")
            
        self._latitude = latitude
        self._longitude = longitude
        
    @property
    def latitude(self):
        return self._latitude
    
    @property
    def longitude(self):
        return self._longitude
    
    
    @property
    def longitude_hemisphere(self):
        return "E" if self.longitude >= 0 else "W"
    
    @property
    def latitude_hemisphere(self):
        return "N" if self.latitude >=0 else "S"
    
    def __repr__(self):
        return f"{typename(self)}(latitude={self.latitude}, longitude={self.longitude})"
    
    def __str__(self):
        
        # use default format_spec of format() to generate the str() representation
        return format(self)
    
    def __format__(self, format_spec):
        # this is the default decimal point format spec
        component_format_spec = ".2f"
        
        # if format_spec contains a dot, then get the suffix and use that for decimal point format
        prefix, dot, suffix = format_spec.partition(".")
        if dot:
            num_decimal_places = int(suffix)
            component_format_spec = f".{num_decimal_places}f"
            print(f"component_format_spec{component_format_spec}")
        longitude = format(abs(self.longitude), component_format_spec)
        latitude = format(abs(self.latitude), component_format_spec)
        return (f"{longitude} {self.longitude_hemisphere}, "
                f"{latitude} {self.latitude_hemisphere}"
               )
    
class EarthPosition(Position):
    pass

class MarsPosition(Position):
    pass
    

def typename(obj):
    return type(obj).__name__

In [10]:
class Location:
    
    def __init__(self, name, position):
        self._name = name
        self._position = position
        
    @property
    def name(self):
        return self._name
    
    @property
    def position(self):
        return self._position
    
    
    def __repr__(self):
        return f"{typename(self)}(name={self.name}, position={self.position})"    
    
    
    def __str__(self):
        return self.name
    
class EarthPosition(Position):
    pass
    


In [11]:
hong_kong = Location("Hong Kong", EarthPosition(22.29, 114.16))
stockholm = Location("Stockholm", EarthPosition(59.33, 18.06))
cape_town = Location("Cap_Town", EarthPosition(-33.93, 18.42))
rotterdam = Location("Rotterdam", EarthPosition(51.96, 4.47))
maracaibo = Location("Maracaibo", EarthPosition(10.65, -71.65))

In [12]:
hong_kong

Location(name=Hong Kong, position=114.16 E, 22.29 N)

In [13]:
print(maracaibo)

Maracaibo


In [26]:
# use class decorator

# a class decorator function receives a class argument
# and returns the class
def auto_repr(cls):
    print(f"Decorating {cls.__name__} with auto_rper")
    members = vars(cls)
    for name, member in members.items():
        print(name, member)
           
    return cls

@auto_repr
class Location:
    
    def __init__(self, name, position):
        self._name = name
        self._position = position
        
    @property
    def name(self):
        return self._name
    
    @property
    def position(self):
        return self._position
    
    
    def __repr__(self):
        return f"{typename(self)}(name={self.name}, position={self.position})"    
    
    
    def __str__(self):
        return self.name
    
class EarthPosition(Position):
    pass
    


Decorating Location with auto_rper
__module__ __main__
__init__ <function Location.__init__ at 0x00000253861DC9D0>
name <property object at 0x0000025386163FB0>
position <property object at 0x0000025386163DD0>
__repr__ <function Location.__repr__ at 0x00000253861DDF30>
__str__ <function Location.__str__ at 0x00000253861DDFC0>
__dict__ <attribute '__dict__' of 'Location' objects>
__weakref__ <attribute '__weakref__' of 'Location' objects>
__doc__ None


### decorate the \_\_repr\_\_ method
* make sure the current class doesn't define the \_\_repr\_\_ method
* the current class has implemented the init() method (check with vars(cls), which doesn't include inherited methods)
* 

In [14]:
# use class decorator

import inspect

# a class decorator function receives a class argument
# and returns the class
def auto_repr(cls):
    members = vars(cls)
   
    # make sure __repr__ has not been implemented
    if "__repr__" in members:
        raise TypeError(f"{cls.__name__} already defines __repr__")
        
    # make sure the current class has an implementation of __init__
    if "__init__" not in members:
        raise TypeError(f"{cls.__name__} does not override __init__")
                        
    # make sure __init__ has all its input arguments except self have
    # the corresponding properties defined
    
    # not calling __init__, but pass the method to the function 
    sig = inspect.signature(cls.__init__) 
    parameter_names = list(sig.parameters)[1:]
        
    if not all(
        isinstance(members.get(name, None), property)
        for name in parameter_names
    ):
        raise TypeError(
        f"Cannot apply auto_repr to {cls.__name__} because not all "
            "__init__ parameters have matching properties"
        )
        
    def synthesized_repr(self):
        return "{typename}({args})".format(
            # we want to use the dynamic class name of self
            typename=typename(self), 
            args=", ".join(
                "{name}={value!r}".format(
                name=name,
                value=getattr(self, name)
                ) for name in parameter_names
            )
        )
    
    # set this method to the class's __repr__ method
    setattr(cls, "__repr__", synthesized_repr)
                        
    return cls

@auto_repr
class Location:
    
    def __init__(self, name, position):
        self._name = name
        self._position = position
        
    @property
    def name(self):
        return self._name
    
    @property
    def position(self):
        return self._position    
    
    
    def __str__(self):
        return self.name
    
class EarthPosition(Position):
    pass
    


In [15]:
hong_kong

Location(name=Hong Kong, position=114.16 E, 22.29 N)

#### Class Decorator Factories

In [60]:
import functools
def postcondition(predicate):
    
    def function_decorator(f):
        
        @functools.wraps(f)
        def wrapper(self, *args, **kwargs):
            result = f(self, *args, **kwargs)
            if not predicate(self):
                raise RuntimeError(
                    f"Post-condition {predicate.__name__} not "
                    f"maintained for {self!r}"
                )
                
            return result 
        return wrapper
    return function_decorator

def at_least_two_locations(itinerary):
    return len(itinerary._locations) >= 2

        
class Itinerary:
        
    @classmethod
    def from_locations(cls, *locations):
        return cls(locations)
    
    @postcondition(at_least_two_locations)
    def __init__(self, locations):
        self._locations = list(locations)
        
    def __str__(self):
        return "\n".join(location.name for location in self._locations)
    
    @property
    def locations(self):
        return tuple(self._locations)
    
    @property
    def origin(self):
        return self._locations[0]
    
    @property
    def destination(self):
        return self._locations[-1]
    
    @postcondition(at_least_two_locations)
    def add(self, location):
        self._locations.append(location)
        
    @postcondition(at_least_two_locations)
    def remove(self, name):
        removal_indexes = [
            index for index, location in enumerate(self._locations)
            if location.name == name
        ]
        
        for index in reversed(removal_indexes):
            del self._locations[index]
            
    @postcondition(at_least_two_locations)
    def truncate_at(self, name):
        stop = None
        for index, location in enumerate(self._locations):
            if location.name == name:
                stop = index + 1
        self._locations = self._locations[:stop]        

In [62]:
trip = Itinerary.from_locations(maracaibo)
trip

RuntimeError: Post-condition at_least_two_locations not maintained for <__main__.Itinerary object at 0x00000205EB2B6C50>

In [61]:
trip = Itinerary.from_locations(maracaibo, rotterdam, stockholm)
trip

In [63]:
print(trip)

Maracaibo
Rotterdam
Stockholm


In [64]:
trip.origin

Location(name=Maracaibo, position=71.65 W, 10.65 N)

In [65]:
trip.destination

Location(name=Stockholm, position=18.06 E, 59.33 N)

In [66]:
trip.add(cape_town)

In [67]:
trip.add(hong_kong)

In [68]:
print(trip)

Maracaibo
Rotterdam
Stockholm
Cap_Town
Hong Kong


In [69]:
trip.remove("Stockholm")

In [70]:
print(trip)

Maracaibo
Rotterdam
Cap_Town
Hong Kong


In [71]:
trip.truncate_at("Rotterdam")

In [72]:
print(trip)

Maracaibo
Rotterdam


In [75]:
import functools
def postcondition(predicate):
    
    def function_decorator(f):
        
        @functools.wraps(f)
        def wrapper(self, *args, **kwargs):
            result = f(self, *args, **kwargs)
            if not predicate(self):
                raise RuntimeError(
                    f"Post-condition {predicate.__name__} not "
                    f"maintained for {self!r}"
                )
                
            return result 
        return wrapper
    return function_decorator

def invariant(predicate):
    function_decorator = postcondition(predicate)
    
    def class_decorator(cls):
        members = list(vars(cls).items())
        for name, member in members:
            if inspect.isfunction(member):
                decorated_member = function_decorator(member)
                setattr(cls, name, decorated_member)
        return cls
    
    return class_decorator

def at_least_two_locations(itinerary):
    return len(itinerary._locations) >= 2

        
@invariant(at_least_two_locations)
class Itinerary:
        
    @classmethod
    def from_locations(cls, *locations):
        return cls(locations)
    
    def __init__(self, locations):
        self._locations = list(locations)
        
    def __str__(self):
        return "\n".join(location.name for location in self._locations)
    
    @property
    def locations(self):
        return tuple(self._locations)
    
    @property
    def origin(self):
        return self._locations[0]
    
    @property
    def destination(self):
        return self._locations[-1]
    
    
    def add(self, location):
        self._locations.append(location)
        
    def remove(self, name):
        removal_indexes = [
            index for index, location in enumerate(self._locations)
            if location.name == name
        ]
        
        for index in reversed(removal_indexes):
            del self._locations[index]
            
    def truncate_at(self, name):
        stop = None
        for index, location in enumerate(self._locations):
            if location.name == name:
                stop = index + 1
        self._locations = self._locations[:stop]        

In [76]:
trip = Itinerary.from_locations(maracaibo)
trip

RuntimeError: Post-condition at_least_two_locations not maintained for <__main__.Itinerary object at 0x00000205ED0DFC70>

In [78]:
trip = Itinerary.from_locations(maracaibo, rotterdam, stockholm)
print(trip)

Maracaibo
Rotterdam
Stockholm


In [79]:
import functools

def no_duplicates(itinerary):
    already_seen = set()
    for location in itinerary._locations:
        if location in already_seen:
            return False
        already_seen.add(location)
    return True

def postcondition(predicate):
    
    def function_decorator(f):
        
        @functools.wraps(f)
        def wrapper(self, *args, **kwargs):
            result = f(self, *args, **kwargs)
            if not predicate(self):
                raise RuntimeError(
                    f"Post-condition {predicate.__name__} not "
                    f"maintained for {self!r}"
                )
                
            return result 
        return wrapper
    return function_decorator

def invariant(predicate):
    function_decorator = postcondition(predicate)
    
    def class_decorator(cls):
        members = list(vars(cls).items())
        for name, member in members:
            if inspect.isfunction(member):
                decorated_member = function_decorator(member)
                setattr(cls, name, decorated_member)
        return cls
    
    return class_decorator

def at_least_two_locations(itinerary):
    return len(itinerary._locations) >= 2

@auto_repr
@invariant(no_duplicates)        
@invariant(at_least_two_locations)
class Itinerary:
        
    @classmethod
    def from_locations(cls, *locations):
        return cls(locations)
    
    def __init__(self, locations):
        self._locations = list(locations)
        
    def __str__(self):
        return "\n".join(location.name for location in self._locations)
    
    @property
    def locations(self):
        return tuple(self._locations)
    
    @property
    def origin(self):
        return self._locations[0]
    
    @property
    def destination(self):
        return self._locations[-1]
    
    
    def add(self, location):
        self._locations.append(location)
        
    def remove(self, name):
        removal_indexes = [
            index for index, location in enumerate(self._locations)
            if location.name == name
        ]
        
        for index in reversed(removal_indexes):
            del self._locations[index]
            
    def truncate_at(self, name):
        stop = None
        for index, location in enumerate(self._locations):
            if location.name == name:
                stop = index + 1
        self._locations = self._locations[:stop]        

In [80]:
trip = Itinerary.from_locations(maracaibo, rotterdam, stockholm)
print(trip)

Maracaibo
Rotterdam
Stockholm


### Summary
* class decorators transform class definitions
* class decorators are unary functions which only accepat a class object as its argument, cls
* class decorators should return a class object, often the same one they accept
* class decorators are a simpler alternative to metaclasses
* class decorator factories facilitate parameterization
* multiple class decorators can be applied